In [1]:
import os

import findspark
findspark.init()

from pyspark.sql import *
from pyspark.sql.types import *
import pyspark.sql.functions as psf
from pyspark.sql.utils import AnalysisException

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.10:0.4.1 pyspark-shell'

In [2]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.session.timeZone', 'UTC')
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [5]:
# paths to read data 
parquets_dir = 'parquets/'
data_dir = 'parquetsmarks/'

In [6]:
# create a list which conatins all the dfs to be unified 
df_list = [spark.read.parquet(parquets_dir + data_dir + path ) \
           for path in os.listdir(parquets_dir + data_dir) if path != ".DS_Store"]

In [8]:
# funtion that returns the union of all the dfs 
def unionAll(*dfs):
    first, rest = dfs[0], dfs[1:]  # Python 3.x, for 2.x you'll have to unpack manually
    return first.sql_ctx.createDataFrame(
        first.sql_ctx._sc.union([df.rdd for df in dfs]),
        first.schema
    )

In [9]:
df_to_write = unionAll(df_list)

AttributeError: 'list' object has no attribute 'sql_ctx'

In [ ]:
# write the parquet
FILENAME = 'film2'
df_to_write.write.parquet(data_dir + FILENAME)